In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [56]:
!pip install matplotlib contextily


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [38]:

import geopandas as gpd
from shapely import wkb
import time
import os
import glob
import duckdb
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from IPython.display import display
import ipywidgets as widgets


# Opprett mappestruktur

In [ ]:
def opprett_mappestruktur(data_mappe="./data"):
    """
    Oppretter nødvendig mappestruktur for konvertering.

    Args:
        data_mappe: Sti til hovedmappen for data

    Returns:
        dict: Stier til opprettede mapper
    """
    # Definer mappestruktur
    rå_mappe = os.path.join(data_mappe, "raw")
    prosessert_mappe = os.path.join(data_mappe, "processed")
    source_mappe = os.path.join(data_mappe, "source")
    incoming_dir_mappe = os.path.join(data_mappe, "incoming")

    # Opprett mapper
    os.makedirs(data_mappe, exist_ok=True)
    os.makedirs(rå_mappe, exist_ok=True)
    os.makedirs(prosessert_mappe, exist_ok=True)
    os.makedirs(source_mappe, exist_ok=True)
    os.makedirs(incoming_dir_mappe, exist_ok=True)

    print(f"Mappestruktur opprettet.")

    # Returner stier for senere bruk
    return {
        "data_mappe": os.path.abspath(data_mappe),
        "rå_mappe": os.path.abspath(rå_mappe),
        "prosessert_mappe": os.path.abspath(prosessert_mappe),
        "source_mappe": os.path.abspath(source_mappe),
        "incoming_mappe": os.path.abspath(incoming_dir_mappe)
    }

def legg_til_tidspartisjoneringskolonner(df, tid_kolonne='date_time_utc'):
    """
    Legger til kolonner for tidspartisjonering (kun time for partisjonering, men beholder dato-kolonner).
    """
    if tid_kolonne not in df.columns:
        print(f"Advarsel: Tidsstempelkolonne '{tid_kolonne}' finnes ikke")
        return df

    if not pd.api.types.is_datetime64_any_dtype(df[tid_kolonne]):
        print(f"Advarsel: Kolonnen '{tid_kolonne}' er ikke en datetime-kolonne")
        return df

    df_med_tid = df.copy()
    df_med_tid['year'] = df_med_tid[tid_kolonne].dt.year
    df_med_tid['month'] = df_med_tid[tid_kolonne].dt.month
    df_med_tid['day'] = df_med_tid[tid_kolonne].dt.day
    df_med_tid['hour'] = df_med_tid[tid_kolonne].dt.hour

    return df_med_tid

def opprett_geodataframe(df):
    """
    Oppretter en GeoDataFrame fra en DataFrame ved å finne koordinater eller geometrikolonner.
    """
    from shapely import wkt
    # Sjekk for lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        try:
            return gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df.longitude, df.latitude),
                crs="EPSG:4326"
            )
        except Exception as e:
            print(f"Kunne ikke opprette geometri fra lat/long: {e}")

    # Sjekk for andre geometrikolonner
    geom_kolonner = [col for col in df.columns if any(
        term in col.lower() for term in ['geom', 'coord', 'point', 'polygon', 'linestring', 'wkt']
    )]

    for col in geom_kolonner:
        if df[col].dtype != 'object':
            continue

        try:
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
        except Exception:
            continue

    return None

def lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner):
    """
    Lagrer en GeoDataFrame som partisjonert GeoParquet.
    """
    from shapely import wkb # NB!! Ikke fjern!

    # Konverterer geometri til WKB for å kunne partisjonere
    df_med_wkb = gdf.copy()
    df_med_wkb['geometry_wkb'] = df_med_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    df_for_partisjon = df_med_wkb.drop(columns=['geometry'])

    # Utfør partisjonering
    df_for_partisjon.to_parquet(målfilsti, partition_cols=partisjon_kolonner)

    # Konverter hver partisjonert fil tilbake til GeoParquet
    konverter_partisjonerte_filer_til_geoparquet(målfilsti)
    return målfilsti

def opprett_år_mappe(år, målfilsti):
    """Oppretter en mappe for et spesifikt år."""
    år_mappe = os.path.join(målfilsti, f"year={år}")
    os.makedirs(år_mappe, exist_ok=True)
    return år_mappe

def forbered_wkb_dataframe(år_df):
    """Forbereder en dataframe med WKB-konvertert geometri for partisjonering."""
    # Konverter geometri til WKB for å kunne partisjonere
    år_df_wkb = år_df.copy()
    år_df_wkb['geometry_wkb'] = år_df_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    år_df_wkb = år_df_wkb.drop(columns=['geometry'])
    return år_df_wkb

def konverter_fil_til_geoparquet(src_file, dst_file):
    """Konverterer en enkelt fil fra WKB-format til GeoParquet."""
    try:
        part_df = pd.read_parquet(src_file)
        if 'geometry_wkb' not in part_df.columns:
            return False

        part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
        part_df = part_df.drop(columns=['geometry_wkb'])
        part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

        # Sørg for at målmappen eksisterer
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        part_gdf.to_parquet(dst_file)
        return True
    except Exception as e:
        print(f"Advarsel: Kunne ikke konvertere {src_file} til GeoParquet: {e}")
        return False

def kopier_og_konverter_filer(temp_år_mappe, år_mappe):
    """Kopierer og konverterer filer fra temp-mappen til målmappen."""
    suksess_count = 0
    feil_count = 0

    # Opprett først alle mappene
    for root, dirs, _ in os.walk(temp_år_mappe):
        for directory in dirs:
            src_dir = os.path.join(root, directory)
            rel_path = os.path.relpath(src_dir, temp_år_mappe)
            dst_dir = os.path.join(år_mappe, rel_path)
            os.makedirs(dst_dir, exist_ok=True)

    # Kopier og konverter filene
    for root, _, files in os.walk(temp_år_mappe):
        for file in files:
            src_file = os.path.join(root, file)
            rel_path = os.path.relpath(src_file, temp_år_mappe)
            dst_file = os.path.join(år_mappe, rel_path)

            if konverter_fil_til_geoparquet(src_file, dst_file):
                suksess_count += 1
            else:
                feil_count += 1

    return suksess_count, feil_count

def konverter_partisjonerte_filer_til_geoparquet(rotmappe):
    """
    Konverterer alle partisjonerte parquet-filer til GeoParquet format.
    """
    from shapely import wkb # NB!! Ikke fjern!
    feil_count = 0

    for root, _, files in os.walk(rotmappe):
        for file in files:
            if not file.endswith('.parquet'):
                continue

            parquet_path = os.path.join(root, file)
            try:
                # Les dataframe
                part_df = pd.read_parquet(parquet_path)

                # Hopp over hvis den ikke har geometry_wkb
                if 'geometry_wkb' not in part_df.columns:
                    continue

                # Konverter WKB tilbake til geometri
                part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
                part_df = part_df.drop(columns=['geometry_wkb'])

                # Lag GeoDataFrame
                part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

                # Skriv GeoParquet-filen
                part_gdf.to_parquet(parquet_path)
            except Exception as e:
                print(f"Feil ved konvertering av {parquet_path}: {e}")
                feil_count += 1

    if feil_count > 0:
        print(f"Advarsel: {feil_count} filer kunne ikke konverteres til GeoParquet")

def konverter_parquet_til_geoparquet(filsti, målfilsti, partisjon_kolonner=None):
    """
    Konverterer parquet-fil til GeoParquet-format med partisjonering.
    Args:
        filsti: Sti til parquet-filen
        målfilsti: Sti hvor GeoParquet-filen skal lagres
        partisjon_kolonner: Liste av kolonnenavn som skal brukes for partisjonering
    """
    try:
        df = pd.read_parquet(filsti)
    except Exception as e:
        print(f"Kunne ikke lese parquet-fil: {e}")
        return False

    # Håndter tidspartisjonering
    tidspartisjonering = False
    if partisjon_kolonner and 'date_time_utc' in partisjon_kolonner:
        partisjon_kolonner.remove('date_time_utc')
        tidspartisjonering = True

    # Legg til alle tidspartisjoneringskolonner, men vi vil bare partisjonere på time
    if tidspartisjonering:
        df = legg_til_tidspartisjoneringskolonner(df)
        # Sett opp partisjonering kun på 'hour'
        partisjon_kolonner = ['hour'] + (partisjon_kolonner or [])

    # Opprett GeoDataFrame
    gdf = opprett_geodataframe(df)
    if gdf is None:
        return False

    # Sjekk at alle partisjoneringskolonner finnes
    if partisjon_kolonner and not all(col in gdf.columns for col in partisjon_kolonner):
        print(f"Advarsel: Ikke alle partisjoneringskolonner finnes i datasettet")
        manglende = [col for col in partisjon_kolonner if col not in gdf.columns]
        print(f"Manglende kolonner: {manglende}")
        return False

    # Lagre med partisjonering (bare time)
    if partisjon_kolonner:
        return lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner)
    else:
        gdf.to_parquet(målfilsti)
        return målfilsti

def konverter_alle_parquet_filer(data_mappe, partisjon_kolonner=None):
    """
    Konverterer alle parquet-filer i en mappe til GeoParquet.
    """
    rå_mappe = os.path.join(data_mappe, "raw")
    prosessert_mappe = os.path.join(data_mappe, "processed")

    # Opprett mapper
    os.makedirs(rå_mappe, exist_ok=True)
    os.makedirs(prosessert_mappe, exist_ok=True)

    resultater = {
        "konvertert": [],
        "feilet": []
    }

    # Finn alle parquet-filer
    parquet_filer = [f for f in os.listdir(rå_mappe)
                     if f.lower().endswith('.parquet') and os.path.isfile(os.path.join(rå_mappe, f))]

    if not parquet_filer:
        print("Ingen parquet-filer funnet i råmappen")
        return resultater

    # Konverter hver fil
    for filnavn in parquet_filer:
        filsti = os.path.join(rå_mappe, filnavn)
        base_filnavn = os.path.splitext(filnavn)[0]
        målfilsti = os.path.join(prosessert_mappe, f"{base_filnavn}.parquet")

        # Konverter filen
        resultat = konverter_parquet_til_geoparquet(
            filsti,
            målfilsti,
            partisjon_kolonner.copy() if partisjon_kolonner else None,
        )

        if resultat:
            resultater["konvertert"].append(filsti)
        else:
            resultater["feilet"].append(filsti)
            print(f"Kunne ikke konvertere: {filsti}")

    return resultater

def vis_partisjoneringsstruktur(konvertert_sti):
    """
    Viser partisjoneringsstrukturen for en konvertert fil.
    """

    for root, dirs, files in os.walk(konvertert_sti, topdown=True, followlinks=False):
        nivå = root.replace(konvertert_sti, "").count(os.sep)
        innrykk = "    " * (nivå + 1)

        # Vis mappenavnet
        mappe_navn = os.path.basename(root)
        if mappe_navn:  # Ikke vis for rot-mappen
            print(f"{innrykk}- {mappe_navn}")

        # Vis antall filer i dypeste mapper
        if not dirs and files:
            print(f"{innrykk}  Inneholder {len(files)} filer")

def vis_datasett_info(filsti):
    """
    Viser informasjon om et GeoParquet datasett.
    """
    try:
        gdf = gpd.read_parquet(filsti)
        print(f"  Datasett størrelse: {len(gdf)} rader, {len(gdf.columns)} kolonner")
        return gdf
    except Exception as e:
        print(f"  Kunne ikke lese filen: {e}")
        return None

def start_konvertering(data_mappe="./data", partisjon_kolonner=None):
    """
    Start konverteringsprosessen fra Parquet til GeoParquet.
    Args:
        data_mappe: Sti til datamappen
        partisjon_kolonner: Liste av kolonner for partisjonering
    """
    start_tid = time.time()
    print(f"Starter konvertering og partisjonering av parquet-filer i {data_mappe} ...\n")
    print(f"Partisjonering vil skje på 'hour' og følgende kolonner: {partisjon_kolonner or []}")

    # Opprett og sjekk mappene
    mappestier = opprett_mappestruktur(data_mappe)
    rå_mappe = mappestier["rå_mappe"]

    # Utfør konverteringen
    resultater = konverter_alle_parquet_filer(data_mappe, partisjon_kolonner)

    print("Konvertering og partisjonering utført:")
    print(f"• Total behandlingstid: {time.time() - start_tid:.2f} sekunder")
    print(f"• Konverterte filer: {len(resultater['konvertert'])}")

    return resultater

## Kjør mappestruktur funksjonen

In [ ]:
mappestier = opprett_mappestruktur()

# Kun konvertering

In [ ]:
# resultater = start_konvertering(data_mappe="./data")

# Manuell konvertering og partisjonering
resultater = start_konvertering(data_mappe="./data", partisjon_kolonner=["date_time_utc", "ship_type"])

## Sett inn verdier og kjør funskjonen for filtrering!

In [57]:
def get_filtered_ais_data_spatial(
    base_path: str,
    ship_type: int,
    center_lat: float,
    center_lon: float,
    buffer_radius_m: float,
    start_date: str,
    start_time: str,
    end_date: str,
    end_time: str,
    max_rows: int
) -> tuple[pd.DataFrame,int]:
    """
    Leser alle parquet-filer under base_path, bruker DuckDB med spatial-utvidelse
    til å filtrere på:
      - ship_type (0 = ingen filtering)
      - tidsrom (dato ± klokkeslett)
      - innenfor buffer rundt et midtpunkt

    Returnerer:
      (df_limited, total_count)
    """
    pattern = os.path.join(base_path, "**", "*.parquet*")
    files = glob.glob(pattern, recursive=True)
    if not files:
        raise IOError(f"Ingen filer funnet for mønster {pattern!r}")

    # Bygg WHERE-klausul
    where = []
    if ship_type != 0:
        where.append(f"ship_type = {ship_type}")

    def make_ts(d, t, is_end=False):
        if d in ("", "0"):
            return None
        if t in ("", "0"):
            t = "23:59:59" if is_end else "00:00:00"
        return f"TIMESTAMP '{d} {t}'"

    ts_start = make_ts(start_date, start_time, is_end=False)
    ts_end   = make_ts(end_date,   end_time,   is_end=True)

    if ts_start and ts_end:
        where.append(f"date_time_utc BETWEEN {ts_start} AND {ts_end}")
    elif ts_start:
        if end_date in ("", "0"):
            if start_time in ("", "0"):
                where.append(f"CAST(date_time_utc AS DATE) = DATE '{start_date}'")
            else:
                where.append(f"date_time_utc >= {ts_start}")
                where.append(f"date_time_utc <= TIMESTAMP '{start_date} 23:59:59'")
        else:
            where.append(f"date_time_utc >= {ts_start}")
    elif ts_end:
        where.append(f"date_time_utc <= {ts_end}")

    # Bufferfiltrering
    radius_deg = buffer_radius_m / 111000.0
    where.append(
        f"ST_Distance(geometry, ST_GeomFromText('POINT({center_lon} {center_lat})')) <= {radius_deg}"
    )

    where_clause = ""
    if where:
        where_clause = " WHERE " + " AND ".join(where)

    con = duckdb.connect()
    con.execute("INSTALL spatial;")
    con.execute("LOAD spatial;")

    # Tell totalt antall treff
    count_sql = f"SELECT COUNT(*) AS cnt FROM read_parquet('{pattern}'){where_clause}"
    total_count = con.execute(count_sql).fetchone()[0]

    # Hent data med LIMIT
    select_sql = (
        f"SELECT * FROM read_parquet('{pattern}')" +
        where_clause +
        " ORDER BY date_time_utc"
    )
    if max_rows and max_rows > 0:
        select_sql += f" LIMIT {max_rows}"

    df = con.execute(select_sql).fetchdf()
    con.close()
    return df, total_count

def render_ais_map(
    df: pd.DataFrame,
    center_lat: float,
    center_lon: float,
    buffer_radius_m: float
) -> folium.Map:
    """
    Tegner buffer og skips-posisjoner som klynger på et Folium-kart.
    """
    m = folium.Map(location=[center_lat, center_lon],
                   zoom_start=12,
                   tiles="cartodb positron")
    folium.Circle(
        location=[center_lat, center_lon],
        radius=buffer_radius_m,
        color='blue',
        fill=True,
        fill_opacity=0.1,
        popup="Bufferområde"
    ).add_to(m)

    cluster = MarkerCluster().add_to(m)
    for _, row in df.iterrows():
        if pd.notnull(row.longitude) and pd.notnull(row.latitude):
            folium.Marker(
                location=[row.latitude, row.longitude],
                popup=(
                    f"MMSI: {row.mmsi}<br>"
                    f"Time: {row.date_time_utc}<br>"
                    f"Lon/Lat: {row.longitude:.4f}, {row.latitude:.4f}"
                ),
                icon=folium.Icon(color='red', icon='info-sign')
            ).add_to(cluster)
    return m

# Filvalg
base_path_label  = widgets.Label("Filbane:")
base_path_widget = widgets.Text(value='./data/processed', layout=widgets.Layout(width='600px'))
load_button      = widgets.Button(description="Last filsti", button_style='info', layout=widgets.Layout(width='150px'))
file_output      = widgets.Output()

# Filterfelt
ship_type_label   = widgets.Label("Ship Type (0 = alle):")
ship_type_widget  = widgets.IntText(value=0, layout=widgets.Layout(width='200px'))

lat_label         = widgets.Label("Koordinat (Lat):")
center_lat_widget = widgets.FloatText(value=58.142359, layout=widgets.Layout(width='200px'))

lon_label         = widgets.Label("Koordinat (Lon):")
center_lon_widget = widgets.FloatText(value=8.025218, layout=widgets.Layout(width='200px'))

radius_label      = widgets.Label("Buffer i meter:")
radius_widget     = widgets.IntText(value=6000, layout=widgets.Layout(width='200px'))

start_date_label  = widgets.Label("Start-dato (YYYY-MM-DD):")
start_date_widget = widgets.Text(value='2025-01-21', layout=widgets.Layout(width='200px'))
start_time_label  = widgets.Label("Start-tid (HH:MM:SS eller 0 for å ignorere):")
start_time_widget = widgets.Text(value='0', layout=widgets.Layout(width='200px'))

end_date_label    = widgets.Label("Slutt-dato (YYYY-MM-DD eller 0 for å ignorere):")
end_date_widget   = widgets.Text(value='0', layout=widgets.Layout(width='200px'))
end_time_label    = widgets.Label("Slutt-tid (HH:MM:SS eller 0 for å ignorere):")
end_time_widget   = widgets.Text(value='0', layout=widgets.Layout(width='200px'))

max_rows_label    = widgets.Label("Maks rader:")
max_rows_widget   = widgets.IntText(value=10, layout=widgets.Layout(width='200px'))

run_button    = widgets.Button(description="Utfør filtrering", button_style='info', layout=widgets.Layout(width='150px'))
run_output    = widgets.Output()

filter_widgets_box = widgets.VBox([
    ship_type_label,   ship_type_widget,
    lat_label,         center_lat_widget,
    lon_label,         center_lon_widget,
    radius_label,      radius_widget,
    start_date_label,  start_date_widget,
    start_time_label,  start_time_widget,
    end_date_label,    end_date_widget,
    end_time_label,    end_time_widget,
    max_rows_label,    max_rows_widget,
    run_button,        run_output
])
filter_widgets_box.layout.display = 'none'

# Vis kun filvalg først
display(widgets.VBox([
    base_path_label, base_path_widget,
    load_button, file_output,
    filter_widgets_box
]))

# --- Handlere ---

def on_load_clicked(b):
    with file_output:
        file_output.clear_output()
        base = base_path_widget.value.strip()
        pattern = os.path.join(base, "**", "*.parquet*")
        files = glob.glob(pattern, recursive=True)
        if not files:
            print(f"Feil: Ingen filer funnet under {base!r}")
            return
        print(f"Fant {len(files)} parquet-filer. Angi filtre under.")
        filter_widgets_box.layout.display = ''

load_button.on_click(on_load_clicked)

def on_run_clicked(b):
    with run_output:
        run_output.clear_output()
        try:
            df, total_count = get_filtered_ais_data_spatial(
                base_path=base_path_widget.value.strip(),
                ship_type=ship_type_widget.value,
                center_lat=center_lat_widget.value,
                center_lon=center_lon_widget.value,
                buffer_radius_m=radius_widget.value,
                start_date=start_date_widget.value.strip(),
                start_time=start_time_widget.value.strip(),
                end_date=end_date_widget.value.strip(),
                end_time=end_time_widget.value.strip(),
                max_rows=max_rows_widget.value
            )
        except Exception as e:
            print("Feil ved henting av data:", e)
            return

        shown = len(df)
        print(f"Hentet totalt {total_count} rader. Viser {shown} rader.")
        display(df)

        m = render_ais_map(
            df,
            center_lat_widget.value,
            center_lon_widget.value,
            radius_widget.value
        )
        display(m)

run_button.on_click(on_run_clicked)
